In [21]:
!pip install xlsxwriter
import json
import xlsxwriter
import io
import requests

from google.colab import auth
#To authenticate with google colab
auth.authenticate_user()

#create service to use google drive
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

#For media upload
from googleapiclient.http import MediaFileUpload

#For unicode shit
try:
    to_unicode = unicode
except NameError:
    to_unicode = str

In [0]:
def downloadGame(number):
    url = 'https://s3-us-west-1.amazonaws.com/riot-developer-portal/seed-data/matches' + str(number) + '.json'
    response = requests.get(url)
    response.encoding = 'latin1'
    games = response.json()
    
    #Cleanes all games by removing unnecessary details
    for i in range(len(games['matches'])):
        del games['matches'][i]['mapId']
        del games['matches'][i]['gameMode']
        del games['matches'][i]['queueId']
        del games['matches'][i]['gameVersion']
        del games['matches'][i]['gameCreation']
        del games['matches'][i]['gameDuration']
        del games['matches'][i]['seasonId']
        
        for y in range(10):
            del games['matches'][i]['participants'][y]['timeline']
            del games['matches'][i]['participantIdentities'][y]['player']['profileIcon']
            del games['matches'][i]['participantIdentities'][y]['player']['platformId']
            del games['matches'][i]['participantIdentities'][y]['player']['matchHistoryUri']
            del games['matches'][i]['participantIdentities'][y]['player']['currentPlatformId']
    
    return games

In [0]:
data = []
matchIds = []
playerNames = []
championPlayed = []
outcome = []
deaths = []
kills = []
wardsPlaced = []
wardsKilled = []
playerTier = []
teamDragons = []
teamBarons = []
minionsKilled = []

In [0]:
for i in range(1,10):
    matches = downloadGame(i)
    
    for x in range(100):
    
    #Above loop goes through every match, bottom loop goes through each player, respectively
        for y in range(10):
            playerNames.append(matches['matches'][x]['participantIdentities'][y]['player']['summonerName'])
            
            
            championPlayed.append(matches['matches'][x]['participants'][y]['championId'])
            
            playerTier.append(matches['matches'][x]['participants'][y]['highestAchievedSeasonTier'])
            deaths.append(matches['matches'][x]['participants'][y]['stats']['deaths'])
            wardsPlaced.append(matches['matches'][x]['participants'][y]['stats']['wardsPlaced'])
            wardsKilled.append(matches['matches'][x]['participants'][y]['stats']['wardsKilled'])
            kills.append(matches['matches'][x]['participants'][y]['stats']['kills'])
            matchIds.append(matches['matches'][x]['gameId'])
            minionsKilled.append(matches['matches'][x]['participants'][y]['stats']['totalMinionsKilled'])
            
            #Find out if the player won/lost and append it to the array
            winner = matches['matches'][x]['participants'][y]['stats']['win']
            if winner == True:
                outcome.append("Win")
            else:
                outcome.append("Loss")
            
            #Find out which team the player was on, and how many barons/dragons the team got    
            playerTeam = matches['matches'][x]['participants'][y]['teamId']
            if playerTeam == 100:
                playerTeam = 0
            else:
                playerTeam = 1
            teamDragons.append(matches['matches'][x]['teams'][playerTeam]['dragonKills'])
            teamBarons.append(matches['matches'][x]['teams'][playerTeam]['baronKills'])

In [0]:
workbook = xlsxwriter.Workbook('largeData.xlsx')
worksheet = workbook.add_worksheet()

In [30]:
worksheet.write('A1', 'Game Id')
worksheet.write('B1', 'Player Name')
worksheet.write('C1', 'Champion Played')
worksheet.write('D1', 'Outcome')
worksheet.write('E1', 'Deaths')
worksheet.write('F1', 'Kills')
worksheet.write('G1', 'Wards Killed')
worksheet.write('H1', 'Wards Placed')
worksheet.write('I1', 'Player Tier')
worksheet.write('J1', 'Team Dragons')
worksheet.write('K1', 'Team Barons')
worksheet.write('L1', 'Minions Killed')

0

In [0]:
data.append(matchIds)
data.append(playerNames)
data.append(championPlayed)
data.append(outcome)
data.append(deaths)
data.append(kills)
data.append(wardsKilled)
data.append(wardsPlaced)
data.append(playerTier)
data.append(teamDragons)
data.append(teamBarons)
data.append(minionsKilled)

In [32]:
worksheet.write_column('A2', data[0])
worksheet.write_column('B2', data[1])
worksheet.write_column('C2', data[2])
worksheet.write_column('D2', data[3])
worksheet.write_column('E2', data[4])
worksheet.write_column('F2', data[5])
worksheet.write_column('G2', data[6])
worksheet.write_column('H2', data[7])
worksheet.write_column('I2', data[8])
worksheet.write_column('J2', data[9])
worksheet.write_column('K2', data[10])
worksheet.write_column('L2', data[11])

0

In [0]:
workbook.close()


In [0]:
file_metadata = {
  'name': 'Large League of legends Data',
  'mimeType': 'text/plain'
}

In [35]:
media = MediaFileUpload('largeData.xlsx', 
                        mimetype='application/vnd.ms-excel',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print 'File ID: %s' % created.get('id')

File ID: 1pvgSQdIOWu1H9lMfnZ1cpTaUmn91AKER
